# Notebook

В рамках блокнота планируется провести EDA данных по ценам на видеокарты. Структура блокнота:
1. Характеристика данных
    1. Датасет с текущими ценами
        1. Описание датасета
        2. Подготовка и очистка данных
        3. Оценка качества данных
    2. Датасет с историческими ценами
2. Анализ данных

In [2]:
import pandas as pd
import matplotlib as mtl
import matplotlib.pyplot as plt
import re
from IPython.display import JSON

gpus_current_df = pd.read_json('data/pcbuilder_gpus_current.json')
gpus_outdated_df = pd.read_csv('data/pcbuilder_gpus_outdated.csv', encoding='latin-1')

%time

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 3.34 µs


## Характеритсика данных

### Описание данных `gpus_outdated_df`

Датасет `gpus_outdated_df` содержит список моделей видеокарт с характеристиками и ценами с Amazon. Датасет собран год назад, так что цены не являются актуальными. Датасет взят с kaggle [GPU Specs and Latest Prices](https://www.kaggle.com/datasets/shilongzhuang/gpu-prices-prediction).

In [37]:
# выводим данные по колонкам для анализа
gpus_outdated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              311 non-null    object
 1   brand             311 non-null    object
 2   model             311 non-null    object
 3   memory            311 non-null    object
 4   memory_interface  311 non-null    object
 5   length            311 non-null    object
 6   interface         311 non-null    object
 7   chipset           311 non-null    object
 8   base_clock        311 non-null    object
 9   clock_speed       311 non-null    object
 10  frame_sync        311 non-null    object
 11  price             311 non-null    object
 12  item_url          311 non-null    object
 13  used              311 non-null    int64 
dtypes: int64(1), object(13)
memory usage: 34.1+ KB



Датасеты содержат 13 столбцов:
- `name`: название продукта;
    - требуется проверка на дубликаты;
- `brand`: название бренда, выпускающего продукт;
    - требуется проверка на дубликаты;
- `model`: конкретная модель видеокарты от производителя;
    - проверка не требуется, т.к. одна модель может быть у разных продуктов; 
- `memory`: объем видеопамяти;
    - требуется проверка на целостность данных
- `memory_interface`: тип видеопамяти;
- `length`: длина видеокарты; 
- `interface`: тип шины для обмена данными с материнской платой;
- `chipset`: строго говоря, это не чипсет, а модель графического процессора, используемого в видеокарте;
- `base_clock`: базовая частота работы процессора (МГц);
- `clock_speed`: максимальная частота работы процессора (МГц);
- `frame_sync`: технология синхронизации кадров;
- `price`: цена на Amazon на момент сбора датасета;
- `item_url`: ссылка на видеокарту; 
- `used`: является ли указанная цена ценой на поддержанную видеокарту;

In [4]:
# пример данных в формате JSON, проверка на наличие скрытых символов в строках
JSON(gpus_outdated_df.head(5).to_dict(orient='records'))

<IPython.core.display.JSON object>

In [99]:
# пример данных
gpus_outdated_df.head(3)

,name,brand,model,memory,memory_interface,length,interface,chipset,base_clock,clock_speed,frame_sync,price,item_url,used
0,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bit HDMI/DP Nvlink Tri-Frozr 2 Ampere Architecture OC Graphics Card (RTX 3090 GAMING X TRIO 24G),MSI,RTX 3090 GAMING X TRIO 24G,24 GB,GDRR6X,335 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1785 MHz,G-Sync,"$1,289.99",https://pcbuilder.net/component-details/graphics-card/msi-rtx-3090-gaming-x-trio-24g-3090gamingxtrio24g/,0
1,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB GDDR6X, iCX3 Technology, ARGB LED, Metal Backplate",EVGA,RTX 3090 FTW3 Ultra,24 GB,GDDR6X,300 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1800 MHz,G-Sync,"$1,199.99",https://pcbuilder.net/component-details/graphics-card/evga-geforce-rtx-3090-24g-p5-3987-kr/,0
2,"ASUS ROG Strix NVIDIA GeForce RTX 3090 White OC Edition Gaming Graphics Card (PCIe 4.0, 24GB GDDR6X, HDMI 2.1, DisplayPort 1.4a",ASUS,ROG Strix RTX 3090 OC Edition,24 GB,GDDR6X,318.5 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1890 MHz,G-Sync,"$1,794.99",https://pcbuilder.net/component-details/graphics-card/asus-rog-strix-rtx-3090-oc-edition-rog-strix-rtx3090-o24g-white/,0


In [40]:
# ищем дубликаты брендов
gpus_outdated_df['brand'].value_counts()

brand
MSI                   67
Gigabyte              45
ASUS                  38
EVGA                  37
ZOTAC                 27
PowerColor            21
XFX                   16
SapphireTechnology    14
Asus                  11
ASRock                 7
NVIDIA                 6
PNY                    5
NekidCow               5
SAPPHIRE               2
VisionTek              2
HIS                    2
Inno3D                 1
ZOTACGAMING            1
Sapphire               1
ASROCK                 1
AMD                    1
DiamondMultimedia      1
Name: count, dtype: int64

В датасете присутствуют следующие дубликаты:
- SapphireTechnology, SAPPHIRE, Sapphire
- ASRock, ASROCK
- ZOTAC, ZOTACGAMING

In [62]:
# проверяем целостность данных clock_speed по паттерну '<any_number> MHz'

mhz_pattern = r'\d+ MHz$'
mask = gpus_outdated_df['clock_speed'].str.contains(mhz_pattern, regex=True)
print(f'unsuitable clock_speed values found: {len(gpus_outdated_df[mask])}')

unsuitable clock_speed values found: 0


В датасет нет неподходящих значений `clock_speed`.

---

Датасет `gpus_current_df` содержит список моделей видеокарт с характеристиками и актуальными ценами с Amazon. Датасет собран с сайта [pcbuilder.net](https://pcbuilder.net/product/graphics-card/). Загрузка осуществлена с помощью скрипта [парсера](../parser/).

In [5]:
gpus_current_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              990 non-null    object 
 1   brand             990 non-null    object 
 2   model             990 non-null    object 
 3   memory            990 non-null    object 
 4   memory_interface  990 non-null    object 
 5   length            990 non-null    object 
 6   interface         990 non-null    object 
 7   chipset           990 non-null    object 
 8   base_clock        990 non-null    object 
 9   clock_speed       990 non-null    object 
 10  frame_sync        990 non-null    object 
 11  price             990 non-null    float64
 12  item_url          990 non-null    object 
dtypes: float64(1), object(12)
memory usage: 100.7+ KB


In [13]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %code_wrap  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %mamba  %man  %matplotlib  %micromamba  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%code_wrap  %%debug  %%file  %%html  %%javascript  %%js  %

In [20]:
%%latex?

Docstring:
Render the cell as a block of LaTeX

The subset of LaTeX which is supported depends on the implementation in
the client.  In the Jupyter Notebook, this magic only renders the subset
of LaTeX defined by MathJax
[here](https://docs.mathjax.org/en/v2.5-latest/tex.html).
File:      ~/courses/dc-ai/hw-1/data-analizer/.venv/lib/python3.10/site-packages/IPython/core/magics/display.py